In [ ]:
!pip install mne pyriemann geoopt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.5 MB 4.7 MB/s 
     |████████████████████████████████| 365 kB 50.9 MB/s 
     |████████████████████████████████| 90 kB 3.3 MB/s 
  Created wheel for pyriemann: filename=pyriemann-0.3-py2.py3-none-any.whl size=78033 sha256=84f4d857244d91c3af842b3c3a03d3dee141e8c55c1e3d18dd4770958a5035cc
  Stored in directory: /root/.cache/pip/wheels/0b/1b/bf/a537f9e17e6c3490004ede419c72f863af1d0d765d25e532ef
Successfully built pyriemann


In [ ]:
import time
import pandas as pd
import numpy as np



#import torch and sklearn
from torch.autograd import Variable
import torch.nn.functional as F
import torch as th
from torch.utils.data.sampler import SubsetRandomSampler
import torch.utils.data
from sklearn.model_selection import StratifiedShuffleSplit


#import util folder
from Geometric_Methods.utils.model import Tensor_CSPNet_Basic
#from utils.optimizers import MixOptimizer
from Geometric_Methods.utils.early_stopping import EarlyStopping
from Geometric_Methods.utils.load_data import load_KU, load_BCIC, dataloader_in_main
from Geometric_Methods.utils.args import args_parser
import Geometric_Methods.utils.geoopt as geoopt

In [ ]:
import argparse

def args_parser():
    parser = argparse.ArgumentParser()

    parser.add_argument('--alg_name', default = 'Tensor_CSPNet', help = 'name of model')
    parser.add_argument('--mlp',      default = False, help = 'whether the classifier is a multiple layer perception or not')

    parser.add_argument('--no-cuda',  action = 'store_true', default=False, help='disables CUDA training')
    parser.add_argument('--initial_lr', type = float, default = 1e-3, help="initial_lr for optimizer")
    parser.add_argument('--decay',      type = float, default = 1, help="decay rate for adjust_learning")

    parser.add_argument('--start_No', type=int, default = 51,  help='testing starts on subject #')
    parser.add_argument('--end_No',   type=int, default = 54,  help='testing ends on subject #')
    parser.add_argument('--epochs',   type=int, default = 50, help='number of epochs to train')
    parser.add_argument('--patience', type=int, default = 10, help='patience for early stopping')

    parser.add_argument('--train_batch_size', type = int, default = 20, help = 'batch size in each epoch for trainning')
    parser.add_argument('--test_batch_size',  type = int, default = 20, help = 'batch size in each epoch for testing')
    parser.add_argument('--valid_batch_size', type = int, default = 20, help = 'batch size in each epoch for validation')

    parser.add_argument('--seed',         type = int, default = 1, metavar='S', help='random seed (default: 1)')
    parser.add_argument('--log_interval', type = int, default = 1, help='how many batches to wait before logging training status')
    parser.add_argument('--save-model', action = 'store_true', default=False, help='for Saving the current Model')

    parser.add_argument('--folder_name',         default = 'results')
    parser.add_argument('--weights_folder_path', default = 'model_paras/')

    args = parser.parse_args(args=[])

    return args

In [ ]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    optimizer.lr = args.initial_lr * (args.decay ** (epoch // 100))


def main(args, train, test, train_y, test_y, sub, total_sub, kf_iter, validation):

    use_cuda = not args.no_cuda and torch.cuda.is_available()
    device   = torch.device("cuda" if use_cuda else "cpu")

    if validation:
        index_split = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)
        for train_index, valid_index in index_split.split(train,train_y):
            train_sampler = SubsetRandomSampler(train_index)
            valid_sampler = SubsetRandomSampler(valid_index)
            
        train       = Variable(torch.from_numpy(train)).double()
        test        = Variable(torch.from_numpy(test)).double()
        train_y     = Variable(torch.LongTensor(train_y))
        test_y      = Variable(torch.LongTensor(test_y))
          
        train_dataset = dataloader_in_main(train, train_y)
        test_dataset  = dataloader_in_main(test, test_y)

        train_kwargs = {'batch_size': args.train_batch_size}
        if use_cuda:
              cuda_kwargs ={'num_workers': 100,
                            'sampler': train_sampler,
                              'pin_memory': True,
                              'shuffle': True     
              }
              train_kwargs.update(cuda_kwargs)
              
        valid_kwargs = {'batch_size': args.valid_batch_size}
        if use_cuda:
              cuda_kwargs ={'num_workers': 10,
                            'sampler':valid_sampler,
                              'pin_memory': True,
                              'shuffle': True     
              }
              valid_kwargs.update(cuda_kwargs)

        test_kwargs = {'batch_size': args.test_batch_size}
        if use_cuda:
              cuda_kwargs ={'num_workers': 1,
                              'pin_memory': True,
                              'shuffle': True      
              }
              test_kwargs.update(cuda_kwargs)

        train_loader  = torch.utils.data.DataLoader(dataset= train_dataset, **train_kwargs)
        valid_loader  = torch.utils.data.DataLoader(dataset= train_dataset, **valid_kwargs)
        test_loader   = torch.utils.data.DataLoader(dataset= test_dataset,  **test_kwargs)
    else:
        train       = Variable(torch.from_numpy(train)).double()
        test        = Variable(torch.from_numpy(test)).double()
        train_y     = Variable(torch.LongTensor(train_y))
        test_y      = Variable(torch.LongTensor(test_y))

        train_dataset = dataloader_in_main(train, train_y)
        test_dataset  = dataloader_in_main(test, test_y)

        train_kwargs  = {'batch_size': args.train_batch_size}

        if use_cuda:
            cuda_kwargs = {'num_workers': 100,
                            'pin_memory': True,
                            'shuffle': True,		
        }
            train_kwargs.update(cuda_kwargs)

        test_kwargs = {'batch_size': args.test_batch_size}
        if use_cuda:
            cuda_kwargs  = {'num_workers': 100,
                            'pin_memory': True,
                            'shuffle': True,      
            }
            test_kwargs.update(cuda_kwargs)

        train_loader  = torch.utils.data.DataLoader(dataset= train_dataset, **train_kwargs)
        test_loader   = torch.utils.data.DataLoader(dataset= test_dataset,  **test_kwargs)


    model = Tensor_CSPNet_Basic(channel_num = train.shape[1]*train.shape[2], 
        mlp = args.mlp,
        ).to(device)

    optimizer = geoopt.optim.RiemannianAdam(model.parameters(), lr=args.initial_lr)

    early_stopping = EarlyStopping(
        alg_name = args.alg_name, 
        path_w   = args.weights_folder_path + args.alg_name + '_checkpoint.pt', 
        patience = args.patience, 
        verbose  = True, 
        )

    print('#####Start Trainning######')

    for epoch in range(1, args.epochs+1):

        adjust_learning_rate(optimizer, epoch)

        model.train()

        train_correct = 0
    
        for batch_idx, (batch_train, batch_train_y) in enumerate(train_loader):

            optimizer.zero_grad()

            logits = model(batch_train.to(device))
            output = F.log_softmax(logits, dim = -1)
            loss   = F.nll_loss(output, batch_train_y.to(device))

            loss.backward()
            optimizer.step()

            if batch_idx % args.log_interval == 0:
                print('----#------#-----#-----#-----#-----#-----#-----')
                pred    = output.data.max(1, keepdim=True)[1]
                train_correct += pred.eq(batch_train_y.to(device).data.view_as(pred)).long().cpu().sum()
                torch.save(model.state_dict(), args.weights_folder_path + args.alg_name+'_model.pth')
                torch.save(optimizer.state_dict(), args.weights_folder_path+'optimizer.pth')


                print('['+args.alg_name+': Sub No.{}/{} Fold {}/10, Epoch {}/{}, Completed {:.0f}%]:\nTrainning loss {:.10f} Acc.: {:.4f}'.format(\
                        sub, total_sub, kf_iter+1, epoch, args.epochs, 100. * (1+batch_idx) / len(train_loader), loss.cpu().detach().numpy(),\
                        train_correct.item()/len(train_loader.dataset)))
                    
        if validation:
            #Validate the Model
            valid_losses  = []
            valid_loss    =  0
            valid_correct =  0

            model.eval()

            for batch_idx, (batch_valid, batch_valid_y) in enumerate(valid_loader):

                logits         = model(batch_valid.to(device))
                output         = F.log_softmax(logits, dim = -1)
                valid_loss    += F.nll_loss(output, batch_valid_y.to(device))
                valid_losses.append(valid_loss.item())
                
                
                pred           = output.data.max(1, keepdim=True)[1]
                valid_correct += pred.eq(batch_valid_y.to(device).data.view_as(pred)).long().cpu().sum()

            print('Validate loss: {:.10f} Acc: {:.4f}'.format(sum(valid_losses), valid_correct.item()/len(valid_loader.dataset)))
            
            early_stopping(np.average(valid_losses), model)
            
            if early_stopping.early_stop:
              print("Early Stopping!")
              break
        else:
            pass
        

    #Testing
    print('###############################################################')
    print('START TESTING')
    print('###############################################################')

    
    model.eval()

    test_loss    = 0
    test_correct = 0

    with torch.no_grad():
        for batch_idx, (batch_test, batch_test_y) in enumerate(test_loader):

            logits        = model(batch_test.to(device))
            output        = F.log_softmax(logits, dim = -1)
            test_loss    += F.nll_loss(output, batch_test_y.to(device))
            
            test_pred     = output.data.max(1, keepdim=True)[1]
            test_correct += test_pred.eq(batch_test_y.to(device).data.view_as(test_pred)).long().cpu().sum()

            print('-----------------------------------')
            print('Testing Batch {}:'.format(batch_idx))
            print('  Pred Label:', test_pred.view(1, test_pred.shape[0]).cpu().numpy()[0])
            print('Ground Truth:', batch_test_y.numpy())


    return test_correct.item()/len(test_loader.dataset), test_loss.item()/len(test_loader.dataset)

In [ ]:
if __name__ == '__main__':

    args   = args_parser()

    alg_df = pd.DataFrame(columns=['R1', 'R2', 'R3','R4', 'R5', 'R6', 'R7', 'R8','R9', 'R10','Avg'])

    print('############Start Task#################')
    
    for sub in range(args.start_No, args.end_No + 1):

        KU_dataset = load_KU(['Dataset/sess1_sub'+str(sub)], 
            'index/KU_index/sess1_sub'+str(sub), 
            alg_name = args.alg_name,
            scenario = 'CV'
            )

        alg_record = []

        start      = time.time()

        for kf_iter in range(0, 10):

            x_train_stack, x_test_stack, y_train, y_test = KU_dataset.generate_training_test_set_CV(kf_iter)

            acc, loss = main(
                args       = args, 
                train      = x_train_stack, 
                test       = x_test_stack, 
                train_y    = y_train, 
                test_y     = y_test,
                kf_iter    = kf_iter, 
                sub        = sub, 
                total_sub  = args.end_No - args.start_No + 1, 
                validation = True,
                )
            

            print('##############################################################')

            print(args.alg_name + ' Testing Loss.: {:4f} Acc: {:4f}'.format(loss, acc))

            alg_record.append(acc)

        end = time.time()

        alg_record.append(np.mean(alg_record))
        
        alg_df.loc[sub] = alg_record
 
        alg_df.to_csv(args.folder_name + '/' \
        + time.strftime("[%Y-%m-%d %H:%M:%S]", time.localtime()) \
        + args.alg_name \
        +'_Sub(' \
        + str(args.start_No) \
        +'-' \
        +str(args.end_No) \
        +')' \
        +'_' \
        + str(args.epochs)\
        + '.csv'\
        , index = False)

############Start Task#################
#####Start Trainning######


/content/drive/My Drive/KoreaUniversity/Geometric_Methods/utils/functional.py:40: UserWarning: torch.eig is deprecated in favor of torch.linalg.eig and will be removed in a future PyTorch release.
torch.linalg.eig returns complex tensors of dtype cfloat or cdouble rather than real tensors mimicking complex tensors.
L, _ = torch.eig(A)
should be replaced with
L_complex = torch.linalg.eigvals(A)
and
L, V = torch.eig(A, eigenvectors=True)
should be replaced with
L_complex, V_complex = torch.linalg.eig(A) (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:3415.)
  s,U[i,j]=th.eig(P[i,j],True); S[i,j]=s[:,0]
/content/drive/My Drive/KoreaUniversity/Geometric_Methods/utils/geoopt/linalg/batch_linalg.py:118: UserWarning: An output with one or more elements was resized since it had shape [1, 20, 20], which does not match the required output shape [1, 1, 20, 20]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero e

----#------#-----#-----#-----#-----#-----#-----
[Tensor_CSPNet: Sub No.51/4 Fold 1/10, Epoch 1/50, Completed 11%]:
Trainning loss 0.6912991030 Acc.: 0.0667
----#------#-----#-----#-----#-----#-----#-----
[Tensor_CSPNet: Sub No.51/4 Fold 1/10, Epoch 1/50, Completed 22%]:
Trainning loss 3.2347939800 Acc.: 0.1000
----#------#-----#-----#-----#-----#-----#-----
[Tensor_CSPNet: Sub No.51/4 Fold 1/10, Epoch 1/50, Completed 33%]:
Trainning loss 3.0356885007 Acc.: 0.1556
----#------#-----#-----#-----#-----#-----#-----
[Tensor_CSPNet: Sub No.51/4 Fold 1/10, Epoch 1/50, Completed 44%]:
Trainning loss 3.3592602635 Acc.: 0.2167
----#------#-----#-----#-----#-----#-----#-----
[Tensor_CSPNet: Sub No.51/4 Fold 1/10, Epoch 1/50, Completed 56%]:
Trainning loss 1.7664504431 Acc.: 0.2833
----#------#-----#-----#-----#-----#-----#-----
[Tensor_CSPNet: Sub No.51/4 Fold 1/10, Epoch 1/50, Completed 67%]:
Trainning loss 0.6066336128 Acc.: 0.3500
----#------#-----#-----#-----#-----#-----#-----
[Tensor_CSPNet: 

/content/drive/My Drive/KoreaUniversity/Geometric_Methods/utils/geoopt/linalg/batch_linalg.py:118: UserWarning: An output with one or more elements was resized since it had shape [1, 20, 20], which does not match the required output shape [1, 1, 20, 20]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  ../aten/src/ATen/native/Resize.cpp:17.)
  return expm(x)


----#------#-----#-----#-----#-----#-----#-----
[Tensor_CSPNet: Sub No.51/4 Fold 2/10, Epoch 1/50, Completed 11%]:
Trainning loss 0.7034672951 Acc.: 0.0556
----#------#-----#-----#-----#-----#-----#-----
[Tensor_CSPNet: Sub No.51/4 Fold 2/10, Epoch 1/50, Completed 22%]:
Trainning loss 4.8256615508 Acc.: 0.1111
----#------#-----#-----#-----#-----#-----#-----
[Tensor_CSPNet: Sub No.51/4 Fold 2/10, Epoch 1/50, Completed 33%]:
Trainning loss 1.3517501594 Acc.: 0.1500
----#------#-----#-----#-----#-----#-----#-----
[Tensor_CSPNet: Sub No.51/4 Fold 2/10, Epoch 1/50, Completed 44%]:
Trainning loss 4.8600538871 Acc.: 0.1944
----#------#-----#-----#-----#-----#-----#-----
[Tensor_CSPNet: Sub No.51/4 Fold 2/10, Epoch 1/50, Completed 56%]:
Trainning loss 5.0408880242 Acc.: 0.2444
----#------#-----#-----#-----#-----#-----#-----
[Tensor_CSPNet: Sub No.51/4 Fold 2/10, Epoch 1/50, Completed 67%]:
Trainning loss 5.8199772009 Acc.: 0.2611
----#------#-----#-----#-----#-----#-----#-----
[Tensor_CSPNet: 

...

----#------#-----#-----#-----#-----#-----#-----
[Tensor_CSPNet: Sub No.54/4 Fold 8/10, Epoch 1/50, Completed 11%]:
Trainning loss 0.6894667298 Acc.: 0.0611
----#------#-----#-----#-----#-----#-----#-----
[Tensor_CSPNet: Sub No.54/4 Fold 8/10, Epoch 1/50, Completed 22%]:
Trainning loss 2.4594011671 Acc.: 0.1222
----#------#-----#-----#-----#-----#-----#-----
[Tensor_CSPNet: Sub No.54/4 Fold 8/10, Epoch 1/50, Completed 33%]:
Trainning loss 1.8989278806 Acc.: 0.1833
----#------#-----#-----#-----#-----#-----#-----
[Tensor_CSPNet: Sub No.54/4 Fold 8/10, Epoch 1/50, Completed 44%]:
Trainning loss 1.2899123098 Acc.: 0.2556
----#------#-----#-----#-----#-----#-----#-----
[Tensor_CSPNet: Sub No.54/4 Fold 8/10, Epoch 1/50, Completed 56%]:
Trainning loss 0.6680856822 Acc.: 0.3111
----#------#-----#-----#-----#-----#-----#-----
[Tensor_CSPNet: Sub No.54/4 Fold 8/10, Epoch 1/50, Completed 67%]:
Trainning loss 1.6013637571 Acc.: 0.3722
----#------#-----#-----#-----#-----#-----#-----
[Tensor_CSPNet: 

/content/drive/My Drive/KoreaUniversity/Geometric_Methods/utils/geoopt/linalg/batch_linalg.py:118: UserWarning: An output with one or more elements was resized since it had shape [1, 20, 20], which does not match the required output shape [1, 1, 20, 20]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  ../aten/src/ATen/native/Resize.cpp:17.)
  return expm(x)


----#------#-----#-----#-----#-----#-----#-----
[Tensor_CSPNet: Sub No.54/4 Fold 9/10, Epoch 1/50, Completed 11%]:
Trainning loss 0.7614076512 Acc.: 0.0333
----#------#-----#-----#-----#-----#-----#-----
[Tensor_CSPNet: Sub No.54/4 Fold 9/10, Epoch 1/50, Completed 22%]:
Trainning loss 4.7912921002 Acc.: 0.0944
----#------#-----#-----#-----#-----#-----#-----
[Tensor_CSPNet: Sub No.54/4 Fold 9/10, Epoch 1/50, Completed 33%]:
Trainning loss 4.1521476955 Acc.: 0.1389
----#------#-----#-----#-----#-----#-----#-----
[Tensor_CSPNet: Sub No.54/4 Fold 9/10, Epoch 1/50, Completed 44%]:
Trainning loss 0.6578013144 Acc.: 0.2167
----#------#-----#-----#-----#-----#-----#-----
[Tensor_CSPNet: Sub No.54/4 Fold 9/10, Epoch 1/50, Completed 56%]:
Trainning loss 2.0718392093 Acc.: 0.2889
----#------#-----#-----#-----#-----#-----#-----
[Tensor_CSPNet: Sub No.54/4 Fold 9/10, Epoch 1/50, Completed 67%]:
Trainning loss 3.0788316086 Acc.: 0.3611
----#------#-----#-----#-----#-----#-----#-----
[Tensor_CSPNet: 

/content/drive/My Drive/KoreaUniversity/Geometric_Methods/utils/geoopt/linalg/batch_linalg.py:118: UserWarning: An output with one or more elements was resized since it had shape [1, 20, 20], which does not match the required output shape [1, 1, 20, 20]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  ../aten/src/ATen/native/Resize.cpp:17.)
  return expm(x)


----#------#-----#-----#-----#-----#-----#-----
[Tensor_CSPNet: Sub No.54/4 Fold 10/10, Epoch 1/50, Completed 11%]:
Trainning loss 0.6977768969 Acc.: 0.0556
----#------#-----#-----#-----#-----#-----#-----
[Tensor_CSPNet: Sub No.54/4 Fold 10/10, Epoch 1/50, Completed 22%]:
Trainning loss 4.5828299525 Acc.: 0.1333
----#------#-----#-----#-----#-----#-----#-----
[Tensor_CSPNet: Sub No.54/4 Fold 10/10, Epoch 1/50, Completed 33%]:
Trainning loss 2.0570045307 Acc.: 0.2056
----#------#-----#-----#-----#-----#-----#-----
[Tensor_CSPNet: Sub No.54/4 Fold 10/10, Epoch 1/50, Completed 44%]:
Trainning loss 3.8059425647 Acc.: 0.2556
----#------#-----#-----#-----#-----#-----#-----
[Tensor_CSPNet: Sub No.54/4 Fold 10/10, Epoch 1/50, Completed 56%]:
Trainning loss 2.4106322104 Acc.: 0.3222
----#------#-----#-----#-----#-----#-----#-----
[Tensor_CSPNet: Sub No.54/4 Fold 10/10, Epoch 1/50, Completed 67%]:
Trainning loss 1.1138581697 Acc.: 0.3778
----#------#-----#-----#-----#-----#-----#-----
[Tensor_CS